In [2]:
import numpy as np 
from joblib import Parallel, delayed
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
import open3d as o3d
from scipy.optimize import minimize
from scipy.linalg import lstsq

path_norm = "pclouds/boxunion_uniform100k_ddist_minmax_layers.normals"
path_curv = "pclouds/armadillo100k_ddist_minmax_layers.curv"
path_pcd = "pclouds/boxunion_uniform100k_ddist_minmax_layers.xyz"

normals = np.loadtxt(path_norm)
points = np.loadtxt(path_pcd)
curvs = np.loadtxt(path_curv)
# print(normals.shape, pcd.shape)    

'''visualization'''

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.normals = o3d.utility.Vector3dVector(normals)
o3d.visualization.draw_geometries([pcd], point_show_normal=True)

'''neighborhood (normal init)'''

def find_neighs(points, k=250):
    nbrs = NearestNeighbors(n_neighbors=k, algorithm= 'kd_tree').fit(points)
    distances, indices = nbrs.kneighbors(points) 
    return distances, indices

def pca_plane(points):
    # print(points.shape)
    pca = PCA(n_components=3)
    pca.fit(points)
    return pca.components_[2]/np.linalg.norm(pca.components_[2])

def scipy_ls(points):
    a = np.hstack((points[:, :2], np.ones(points.shape[0]).reshape(-1, 1)))
    b= points[:, 2]
    abc, residue, rank, s = lstsq(a, b) 
    return abc/np.linalg.norm(abc)

def plane_cost(params, points):
    a, b, c, d = params
    # Compute the signed distances from each point to the plane
    distances = np.abs(a * points[:, 0] + b * points[:, 1] + c * points[:, 2] + d) / np.sqrt(a**2 + b**2 + c**2)
    # Return the sum of squared distances as the cost
    return np.sum(distances**2)

def scipy_min(points):
    ini_guess= np.array([1.0,1.0,1.0,1.0])
    res = minimize(plane_cost, ini_guess, args=(points), method='Nelder-Mead', tol=1e-6)
    return res.x[:3]/np.linalg.norm(res.x[:3])

dists, indices = find_neighs(points, 250)
# print(pcd[indices].shape)
neighs = points[indices]

norm_esti_pca = Parallel(n_jobs=-1)(delayed(pca_plane)(point_neighs) for point_neighs in neighs)
# norm_esti_ls = Parallel(n_jobs=-1)(delayed(scipy_ls)(point_neighs) for point_neighs in neighs)
# norm_esti_min = Parallel(n_jobs=-1)(delayed(scipy_min)(point_neighs) for point_neighs in neighs)



Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [13]:
print(norm_esti_pca[0], norm_esti_pca[1])
print(normals[0], normals[1])
print(np.dot(norm_esti_pca[0], normals[0]), np.dot(norm_esti_pca[1], normals[1]))

[ 0. -1.  0.] [-0.23809519  0.28105998  0.92968595]
[ 0.  1. -0.] [-0.  0. -1.]
-1.0 -0.9296859511003669


In [5]:
np.diag(np.dot(norm_esti_pca[:60000] , normals[:60000].T))

In [19]:
memory_size_bytes = 400000
memory_size_gb = memory_size_bytes / (1024 ** 3)

print(f"Memory size: {memory_size_gb:.2f} GB")


Memory size: 0.00 GB


In [10]:

norm_= normals/np.linalg.norm(normals, axis=1)[:,np.newaxis]
dot_pca = np.abs(np.diag(np.dot(norm_esti_pca[:2048] , norm_[:2048,:].T))).mean()
# dot_ls = np.abs(np.diag(np.dot(norm_esti_ls[:300] , norm_[:300,:].T))).mean()
# dot_min = np.abs(np.diag(np.dot(norm_esti_min[:300] , norm_[:300,:].T))).mean()

# print(dot_pca, dot_ls, dot_min)
print(dot_pca)

0.9642755525983011


In [8]:
type(norm_)

numpy.ndarray

In [4]:
import os

    # Get a list of all files in the folder
files = os.listdir("pclouds")

# Create a dictionary to store file lists based on extensions
file_dict = {}

# Iterate over each file and categorize based on extension
for file in files:
    filename, extension = os.path.splitext(file)
    print(filename, extension)
    break
    

armadillo100k_noise_brown_4.00e-02 .curv
Boxy_smooth100k_noise_white_1.00e-02 .pidx
armadillo100k_noise_white_8.00e-02 .xyz
sheet_analytic100k_noise_white_1.00e-02 .pidx
fandisk100k_noise_brown_5.00e-02 .xyz
sphere_analytic100k_noise_white_8.00e-02 .xyz
netsuke100k_noise_white_7.00e-02 .normals
galera100k_noise_white_1.00e-01 .curv
sphere100k_ddist_minmax_layers .curv
boxunion2100k_noise_white_5.00e-02 .normals
armadillo100k_noise_white_1.00e-02 .curv
icosahedron100k_ddist_minmax_layers .normals
boxunion2100k_noise_white_8.00e-02 .xyz
star_smooth100k_noise_white_7.00e-02 .curv
star_smooth100k_noise_white_1.00e-01 .pidx
pipe100k_ddist_minmax .xyz
Cup34100k_ddist_minmax_layers .normals
galera100k_ddist_minmax .pidx
cylinder_analytic100k_noise_white_3.00e-02 .normals
flower100k_noise_white_9.00e-02 .xyz
flower100k_noise_white_4.00e-02 .curv
icosahedron100k_noise_white_2.00e-02 .normals
tortuga100k_ddist_minmax_layers .pidx
boxunion2100k .curv
boxunion_uniform100k .curv
Liberty100k_noise_w

In [16]:
import resource

# Get the maximum resident set size in kilobytes
max_memory_kb = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss

# Convert kilobytes to gigabytes
max_memory_gb = max_memory_kb / (1024 * 1024)

print(f"Maximum memory usage: {max_memory_gb:.2f} GB")


Maximum memory usage: 2.26 GB


In [ ]:
#kill kernel
import os
os._exit(00)


: 

In [10]:
import os
import numpy as np
from natsort import natsorted
from glob import glob

def load_pcds(path):
    files = glob(os.path.join(path, "*.xyz"))
    print (files)
    # pcds = np.array([np.loadtxt(file) for file in files])
    # return pcds
x = load_pcds("pclouds")


['pclouds/armadillo100k_noise_white_8.00e-02.xyz', 'pclouds/fandisk100k_noise_brown_5.00e-02.xyz', 'pclouds/sphere_analytic100k_noise_white_8.00e-02.xyz', 'pclouds/boxunion2100k_noise_white_8.00e-02.xyz', 'pclouds/pipe100k_ddist_minmax.xyz', 'pclouds/flower100k_noise_white_9.00e-02.xyz', 'pclouds/Liberty100k_noise_white_1.00e-01.xyz', 'pclouds/netsuke100k_ddist_minmax_layers.xyz', 'pclouds/boxunion2100k_ddist_minmax.xyz', 'pclouds/sphere100k_noise_white_1.00e-01.xyz', 'pclouds/sphere_analytic100k_noise_white_3.00e-02.xyz', 'pclouds/sphere_analytic100k.xyz', 'pclouds/armadillo100k_noise_brown_1.00e-02.xyz', 'pclouds/cylinder_analytic100k_noise_white_2.00e-02.xyz', 'pclouds/icosahedron100k_noise_white_7.00e-02.xyz', 'pclouds/armadillo100k_noise_brown_4.00e-02.xyz', 'pclouds/icosahedron100k_noise_white_9.00e-02.xyz', 'pclouds/column_head100k_ddist_minmax_layers.xyz', 'pclouds/cylinder100k_ddist_minmax_layers.xyz', 'pclouds/Cup33100k_noise_white_1.00e-02.xyz', 'pclouds/netsuke100k_noise_wh

In [1]:
import os
from natsort import natsorted
import glob

# Define the folder path
pc_folder_path = 'pclouds'  # Change this to the actual path

# Find all file paths ending with .xyz
xyz_files = natsorted(glob.glob(os.path.join(pc_folder_path, '*.xyz')))

# Find all file paths ending with .normals
normals_files = natsorted(glob.glob(os.path.join(pc_folder_path, '*.normals')))

print(len(xyz_files), len(normals_files))

# Print or use the results as needed
# print("XYZ Files:")
# for xyz_file in xyz_files:
#     print(xyz_file)

# print("\nNormals Files:")
# for normals_file in normals_files:
#     print(normals_file)


383 381


In [2]:
c=0
ref_xyz = []
ref_normals = []
for xyz,norm in zip(xyz_files, normals_files):
    xyz_name,_ = os.path.splitext(xyz)
    norm_name,_ = os.path.splitext(norm)
    if f"{xyz_name}.normals" not in normals_files or f"{norm_name}.xyz" not in xyz_files:
        print(xyz_name, norm_name)
        c=c+1
        continue
    else:
        ref_xyz.append(xyz)
        ref_normals.append(norm)
    # print(xyz_name, norm_name)
    # if xyz_name != norm_name:
    #     c=c+1
    #     print(xyz_name, norm_name)
print(c)
    

pclouds/nyu_4 pclouds/pipe100k
pclouds/nyu_4_whitenoise pclouds/pipe100k_ddist_minmax
2


In [5]:
print("pclouds/pipe100k_ddist_minmax_layers.xyz" in ref_xyz)

True


In [11]:
len(ref_normals)

379

In [12]:
ref_xyz = natsorted(ref_xyz)
ref_normals = natsorted(ref_normals)
len(ref_xyz)

379

In [4]:
c=0

for xyz,norm in zip(ref_xyz, ref_normals):
    xyz_name,_ = os.path.splitext(xyz)
    norm_name,_ = os.path.splitext(norm)
    # print(xyz_name, norm_name)
    if xyz_name != norm_name:
        c=c+1
        print(xyz_name, norm_name)
print(c)

pclouds/pipe100k pclouds/pipe100k_ddist_minmax_layers
pclouds/pipe100k_ddist_minmax pclouds/pipe100k_noise_white_1.00e-01
pclouds/pipe100k_ddist_minmax_layers pclouds/pipe100k_noise_white_1.00e-02
pclouds/pipe100k_noise_white_1.00e-01 pclouds/pipe100k_noise_white_2.00e-02
pclouds/pipe100k_noise_white_1.00e-02 pclouds/pipe100k_noise_white_3.00e-02
pclouds/pipe100k_noise_white_2.00e-02 pclouds/pipe100k_noise_white_4.00e-02
pclouds/pipe100k_noise_white_3.00e-02 pclouds/pipe100k_noise_white_5.00e-02
pclouds/pipe100k_noise_white_4.00e-02 pclouds/pipe100k_noise_white_6.00e-02
pclouds/pipe100k_noise_white_5.00e-02 pclouds/pipe100k_noise_white_7.00e-02
pclouds/pipe100k_noise_white_6.00e-02 pclouds/pipe100k_noise_white_8.00e-02
pclouds/pipe100k_noise_white_7.00e-02 pclouds/pipe100k_noise_white_9.00e-02
pclouds/pipe100k_noise_white_8.00e-02 pclouds/pipe_curve100k
pclouds/pipe100k_noise_white_9.00e-02 pclouds/pipe_curve100k_ddist_minmax
pclouds/pipe_curve100k pclouds/pipe_curve100k_ddist_minmax_l